In [2]:
from IPython.display import clear_output
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install pytorch_pretrained_bert
!pip install ekphrasis
!wget https://raw.githubusercontent.com/marcopoli/AlBERTo-it/master/vocab.txt
!git clone https://github.com/gioanat/progettoML
clear_output()

#!ln -ls progettoML/generate_utils.py generate_utils.oy # if import does not work
from progettoML.generate_utils import GenerateHints
from progettoML.generate_utils import parse_hashtags, AlBERTo_Preprocessing
obj = GenerateHints()
clear_output()

In [7]:
def run_server():
  import os
  os.system('wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip')
  os.system('unzip ngrok-stable-linux-amd64.zip')
  os.system('tensorboard --logdir /content/trainingdata/objectdetection/ckpt_output/trainingImatges/ --host 0.0.0.0 --port 6006 &')
  os.system('./ngrok http 6006 &')

  import subprocess
  output = None
  while output is None:
    try:
      output = subprocess.check_output('curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"', shell=True)
      print(output.decode('utf-8'))
    except:
      print("Retrying..")

print("Inserisci l'url nell'applicazione ed esegui la cella sottostante:")
run_server()

Inserisci l'url nell'applicazione ed esegui la cella sottostante:
http://0a9ef6e6c50c.ngrok.io



In [ ]:
from flask import Flask, request, jsonify
from werkzeug.wrappers import Request, Response
!pip install flask-cors
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/", methods=['GET'])
def health_check():
  return "OK"

@app.route("/lang", methods=['POST'])
def set_lang():
  data = request.get_json()['language']
  isItalian = data == 'it'
  print("Language set to", data, isItalian)
  set_language(isItalian)
  return "OK"

@app.route("/", methods=['POST'])
def hello():
  data = request.get_json()
  if data['whoami'] == 'Gioana':
      return "Ti amo tanto ❤️"
  return "Chi sei?"

@app.route("/generateTweets", methods=['POST'])
def generate_tweets():
  seed = request.get_json()['seed']
  ## Aggiungere parole
  phrases = obj.generate_phrase(seed, num_phrase = 3)
  return jsonify([parse_hashtags(phrase) for phrase in phrases])

@app.route("/generateWords", methods=['POST'])
def generate_response_hints():
  seed = request.get_json()['seed']
  hints = obj.generate_hints(seed, num_hints = 3)
  return jsonify(hints)

@app.route("/translate", methods=['POST'])
def translate():
  seed = request.get_json()['seed']
  a = AlBERTo_Preprocessing(do_lower_case=True)
  b = a.preprocess(seed)
  new_text = obj.traslated_funct(b)
  return parse_hashtags(new_text)

from werkzeug.serving import run_simple
run_simple('localhost', 6006, app)

 * Running on http://localhost:6006/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jun/2021 13:22:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2021 13:22:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2021 13:22:56] "OPTIONS /generateTweets HTTP/1.1" 200 -


ciao
Batch 0
Batch 1
Batch 2


127.0.0.1 - - [20/Jun/2021 13:23:31] "POST /generateTweets HTTP/1.1" 200 -


Finished batch 1 in 35.319s
['[CLS]', 'ciao', 'vita', 'bella', 'by', 'max', '##la', 'tu', 'sei', 'una', 'di', '##ne', 'i', 'la', 'l', '?']
['[CLS]', 'ciao', 'tu', 'milano', 'a', 'mi', 'mia', 'mamma', 'el', '##u', 'l', 'mio', 'futuro', 'sei', 'tu', 'mi', 'stai', 'distruggendo', 'il', 'cuore', '<', 'ur', '##l', '>', 'la', 'voglia', 'di', 'un', 'buon', 'inizio', 'a', 'settimana', 'prima', 'della', 'partita', 'e', 'tanta', '!']
['[CLS]', 'ciao', 'son', 'tutti', '/', 'me', 'le', '##le', 'la', 'mia', 'ragazza', 'el', '##a', 'il', 'la', '##n', 'el', '##a', 'il', '##e', 'al', '##o', 'e', 'una', 'delle', 'piu', 'belle', 'ragazze', 'anche', 'loro', '<', 'us', '##er', '>', 'grazie', 'a', 'tee', 'e', 'buona', 'giornata', 'a', 'presto', '[SEP]']
